In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, log_loss, precision_recall_curve,\
roc_curve, roc_auc_score, confusion_matrix

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

**Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта.
При обучении моделей обязательно использовать кроссвалидацию**

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [5]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [6]:
feats = FeatureUnion(continuos_transformers + cat_transformers + base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [7]:
%%time
start_log = time.time()
classifier_log = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_log, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier_log.fit(X_train, y_train)
y_score_log = classifier_log.predict_proba(X_test)[:, 1]

end_log = time.time()
time_log = end_log - start_log

CV score is 0.7867401104915408+-0.00852135511666111
CPU times: user 8.06 s, sys: 113 ms, total: 8.17 s
Wall time: 2.2 s


In [8]:
%%time
start_bayes = time.time()
classifier_bayes = Pipeline([
    ('features',feats),
    ('classifier', GaussianNB()),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_bayes, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier_bayes.fit(X_train, y_train)
y_score_bayes = classifier_bayes.predict_proba(X_test)[:, 1]

end_bayes = time.time()
time_bayes = end_bayes - start_bayes

CV score is 0.6902821520635449+-0.011556054941538583
CPU times: user 1.33 s, sys: 51.5 ms, total: 1.38 s
Wall time: 918 ms


In [9]:
%%time
start_rf = time.time()
classifier_rf = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_rf, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier_rf.fit(X_train, y_train)
y_score_rf = classifier_rf.predict_proba(X_test)[:, 1]

end_rf = time.time()
time_rf = end_rf - start_rf

CV score is 0.7734501681056019+-0.007171140345435727
CPU times: user 1min 4s, sys: 562 ms, total: 1min 5s
Wall time: 1min 5s


In [10]:
%%time
start_ada = time.time()
classifier_ada = Pipeline([
    ('features',feats),
    ('classifier', AdaBoostClassifier(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_ada, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier_ada.fit(X_train, y_train)
y_score_ada = classifier_ada.predict_proba(X_test)[:, 1]

end_ada = time.time()
time_ada = end_ada - start_ada

CV score is 0.7952419477681905+-0.0071441451920620375
CPU times: user 16.7 s, sys: 12.4 ms, total: 16.7 s
Wall time: 16.7 s


вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [11]:
metrics = {}
precision_list = []
recall_list = []
fscore_list = []
rocauc_list = []
time_list = []

In [12]:
b = 1
precision_log, recall_log, thresholds_log = precision_recall_curve(y_test.values, y_score_log)
fscore_log = (1 + b ** 2) * (precision_log * recall_log) / (b ** 2 * precision_log + recall_log)
# locate the index of the largest f score
ix = np.argmax(fscore_log)
# print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_log[ix], 
#                                                                         fscore_log[ix],
#                                                                         precision_log[ix],
#                                                                         recall_log[ix]))
roc_auc_score_log = roc_auc_score(y_true=y_test, y_score=classifier_log.predict_proba(X_test)[:,1])
log_loss_log = log_loss(y_true=y_test, y_pred=classifier_log.predict_proba(X_test)[:,1])
# print(f'ROC_AUC score: {roc_auc_score_log}\nLog_loss score: {log_loss_log}\n')

precision_list.append(precision_log[ix])
recall_list.append(recall_log[ix])
fscore_list.append(fscore_log[ix])
rocauc_list.append(roc_auc_score_log)
time_list.append(time_log)

Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838
ROC_AUC score: 0.7840347790421852
Log_loss score: 0.5779604008230662



In [13]:
b = 1
precision_bayes, recall_bayes, thresholds_bayes = precision_recall_curve(y_test.values, y_score_bayes)
fscore_bayes = (1 + b ** 2) * (precision_bayes * recall_bayes) / (b ** 2 * precision_bayes + recall_bayes)
# locate the index of the largest f score
ix = np.argmax(fscore_bayes)
# print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_bayes[ix], 
#                                                                         fscore_bayes[ix],
#                                                                         precision_bayes[ix],
#                                                                         recall_bayes[ix]))
roc_auc_score_bayes = roc_auc_score(y_true=y_test, y_score=classifier_bayes.predict_proba(X_test)[:,1])
log_loss_bayes = log_loss(y_true=y_test, y_pred=classifier_bayes.predict_proba(X_test)[:,1])
# print(f'ROC_AUC score: {roc_auc_score_bayes}\nLog_loss score: {log_loss_bayes}\n')

precision_list.append(precision_bayes[ix])
recall_list.append(recall_bayes[ix])
fscore_list.append(fscore_bayes[ix])
rocauc_list.append(roc_auc_score_bayes)
time_list.append(time_bayes)

Best Threshold=0.048964, F-Score=0.686, Precision=0.563, Recall=0.878
ROC_AUC score: 0.6876309798112793
Log_loss score: 1.1424022858388785



In [14]:
b = 1
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(y_test.values, y_score_rf)
fscore_rf = (1 + b ** 2) * (precision_rf * recall_rf) / (b ** 2 * precision_rf + recall_rf)
# locate the index of the largest f score
ix = np.argmax(fscore_rf)
# print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_rf[ix], 
#                                                                         fscore_rf[ix],
#                                                                         precision_rf[ix],
#                                                                         recall_rf[ix]))
roc_auc_score_rf = roc_auc_score(y_true=y_test, y_score=classifier_rf.predict_proba(X_test)[:,1])
log_loss_rf = log_loss(y_true=y_test, y_pred=classifier_rf.predict_proba(X_test)[:,1])
# print(f'ROC_AUC score: {roc_auc_score_rf}\nLog_loss score: {log_loss_rf}\n')

precision_list.append(precision_rf[ix])
recall_list.append(recall_rf[ix])
fscore_list.append(fscore_rf[ix])
rocauc_list.append(roc_auc_score_rf)
time_list.append(time_rf)

Best Threshold=0.350000, F-Score=0.719, Precision=0.643, Recall=0.816
ROC_AUC score: 0.7710366181802983
Log_loss score: 0.5992984853728378



In [15]:
b = 1
precision_ada, recall_ada, thresholds_ada = precision_recall_curve(y_test.values, y_score_ada)
fscore_ada = (1 + b ** 2) * (precision_ada * recall_ada) / (b ** 2 * precision_ada + recall_ada)
# locate the index of the largest f score
ix = np.argmax(fscore_ada)
# print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_ada[ix], 
#                                                                         fscore_ada[ix],
#                                                                         precision_ada[ix],
#                                                                         recall_ada[ix]))
roc_auc_score_ada = roc_auc_score(y_true=y_test, y_score=classifier_ada.predict_proba(X_test)[:,1])
log_loss_ada = log_loss(y_true=y_test, y_pred=classifier_ada.predict_proba(X_test)[:,1])
# print(f'ROC_AUC score: {roc_auc_score_ada}\nLog_loss score: {log_loss_ada}\n')

precision_list.append(precision_ada[ix])
recall_list.append(recall_ada[ix])
fscore_list.append(fscore_ada[ix])
rocauc_list.append(roc_auc_score_ada)
time_list.append(time_ada)

Best Threshold=0.497430, F-Score=0.738, Precision=0.692, Recall=0.789
ROC_AUC score: 0.7945722371129712
Log_loss score: 0.6869766413646277



In [16]:
metrics['precision'] = precision_list
metrics['recall'] = recall_list
metrics['f-score'] = fscore_list
metrics['ROC AUC'] = rocauc_list
metrics['time'] = time_list

In [17]:
df = pd.DataFrame(data=metrics, index=['LogisticRegression',
                                       'GaussianNB',
                                       'RandomForestClassifier',
                                       'AdaBoostClassifier'])
df

,precision,recall,f-score,ROC AUC,time
LogisticRegression,0.647431,0.837558,0.730323,0.784035,2.203690
GaussianNB,0.562726,0.877995,0.685866,0.687631,0.918153
RandomForestClassifier,0.642669,0.815553,0.718863,0.771037,65.432626
AdaBoostClassifier,0.692471,0.789401,0.737766,0.794572,16.672726


**Сделать выводы о том, какая модель справилась с задачей лучше других**

AdaBoost и LogisticRegression показали примерно одинаковые результаты, но при этом скорость обучения AdaBoost оказалась в 8 раз медленнее, поэтому LogisticRegression показала оптимальное сочетание метрик и скорости